In [21]:
import requests
import re
import json
import pandas as pd
from bs4 import BeautifulSoup
import os
import time
import random

In [4]:
data_path = '/home/bld/data/data3/peng/Bridge/Beijing/北京市法规规章规范性文件数据库.xlsx'
data = pd.read_excel(data_path)
data

,标题,标题链接,标签,信息,信息1,信息2
0,北京市实施《中华人民共和国动物防疫法》办法,https://www.beijing.gov.cn/zhengce/dfxfg/20240...,北京市,[发文字号],[发文机构] 北京市人民代表大会常务委员会,[发布日期] 2024-07-26
1,北京市人力资源和社会保障局关于2024年积分落户公示及落户办理有关工作的通告,https://www.beijing.gov.cn/zhengce/gfxwj/sj/20...,北京市,[发文字号] 京人社发〔2024〕11号,[发文机构] 北京市人力资源和社会保障局,[发布日期] 2024-07-15
2,北京市人力资源和社会保障局等部门关于印发《北京市加快数字人才培育支撑数字经济发展实施方案（2...,https://www.beijing.gov.cn/zhengce/gfxwj/sj/20...,北京市,[发文字号] 京人社专技字〔2024〕104号,[发文机构] 北京市人力资源和社会保障局,[发布日期] 2024-07-11
3,北京市民政局关于印发《北京市街道（乡镇）区域养老服务中心内设社区餐厅建设工作指引》的通知,https://www.beijing.gov.cn/zhengce/gfxwj/sj/20...,北京市,[发文字号] 京民养老发〔2024〕74号,[发文机构] 北京市民政局,[发布日期] 2024-07-08
4,北京市司法局关于参加2023年国家统一法律职业资格考试成绩合格的2024年应届毕业生等在京申...,https://www.beijing.gov.cn/zhengce/gfxwj/sj/20...,北京市,[发文字号],[发文机构] 北京市司法局,[发布日期] 2024-07-05
...,...,...,...,...,...,...
6320,关于召开整顿社会秩序会议情况的报告,https://www.beijing.gov.cn/zhengce/zfwj/zfwj/b...,北京市,[发文字号] 京政办发〔1980〕6号,[发文机构],[发布日期] 1980-01-09
6321,北京市人民政府办公厅 转发教育部等六部委《关于拟改变 对出国留学人员、访问学者所得奖学金 和...,https://www.beijing.gov.cn/zhengce/zfwj/zfwj/b...,北京市,[发文字号] 京政办发〔1980〕5号,[发文机构],[发布日期] 1980-01-09
6322,北京市人民政府办公厅转发《财政部、外交部、国务院办公室关于修订三项外事费用开支标准的通知》,https://www.beijing.gov.cn/zhengce/zfwj/zfwj/b...,北京市,[发文字号] 京政办发〔1980〕1号,[发文机构] 北京市人民政府办公厅,[发布日期] 1980-01-07
6323,关于将北京市公共交通局 改为北京市公共交通总公司的通知,https://www.beijing.gov.cn/zhengce/zfwj/zfwj/s...,北京市,[发文字号] 京政发〔1980〕38号,[发文机构],[发布日期] 1980-01-01


In [5]:
headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    'Cache-Control': 'no-cache',
    'Connection': 'keep-alive',
    'Pragma': 'no-cache',
    'Referer': 'http://120.35.29.173:8411/document-detail?id=44c601daca678ce368378ecbe69907ec&searchValue=',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36 Edg/128.0.0.0',
}

In [45]:
i = 0
policy_data = []
start_index = 1975
# 遍历每个标题链接
for index, row in data.iterrows():
    i += 1
    # # 只提取前5行
    # if i > 3:
    #     break
    # 跳过前1969条数据
    if i < start_index:
        continue
    print(f"-------------------------------------------第{i}条数据----------------------------------------")
    url = row['标题链接']
    time.sleep(random.uniform(0, 1))
    response = requests.get(url, headers=headers)
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')
    # 查找包含信息的 <ol> 标签
    container_div = soup.find('div', class_='container')
    if container_div:
        title_info = container_div.find('ol')
    else:
        title_info = None
    # 提取标题
    title = soup.find('h1').text.strip() if soup.find('h1') else soup.find('title').text.strip()
    # print(f"标题：{title}")
    # 提取结构信息
    def extract_info_by_index(index):
        if title_info:
            li_tags = title_info.find_all('li')
            if index < len(li_tags):
                span_tag = li_tags[index].find('span')
                return span_tag.text.strip() if span_tag else 'N/A'
        return 'N/A'

    theme = extract_info_by_index(0)
    issuing_agency = extract_info_by_index(1)
    joint_issuing_agency = extract_info_by_index(2)
    implementation_date = extract_info_by_index(3)
    date_formed = extract_info_by_index(4)
    document_number_1 = extract_info_by_index(5)
    abolition_date = extract_info_by_index(6)
    date_published = extract_info_by_index(7)
    validity = extract_info_by_index(8)
    
    file_source_tag = title_info.find_all('li')[9] if title_info and len(title_info.find_all('li')) > 8 else None
    file_source = file_source_tag.find('span').text.strip() if file_source_tag and file_source_tag.find('span') else 'N/A'
    
    # 发文字号
    document_number = ""
    if title_info:
        fwzhdz = title_info.find('span', class_='fwzhdz')
        fwzhnf = title_info.find('span', class_='fwzhnf')
        fwzhxh = title_info.find('span', class_='fwzhxh')
        if fwzhdz and fwzhnf and fwzhxh:
            document_number = f"{fwzhdz.text.strip()}{fwzhnf.text.strip()}{fwzhxh.text.strip()}"
    # 打印提取的信息
    print(f'标题：{title}',f"主题分类：{theme}",f"发文机构：{issuing_agency}",f"联合发文机构：{joint_issuing_agency}",f"实施日期：{implementation_date}",f"形成日期：{date_formed}",f"废止日期：{abolition_date}",f'发文字号:{document_number_1}',f"发布日期：{date_published}",f"有效性：{validity}",f"文件来源：{file_source}",f"发文字号：{document_number}")

    # 查找包含政策文本内容的 <div> 标签
    content_div = soup.find('div', class_='mainTextBox')
    if content_div:
        main_text_div = content_div.find('div', id='mainText')
        if main_text_div:
            policy_text = main_text_div.get_text(separator='\n').strip()
        else:
            policy_text = 'N/A'
    else:
        policy_text = 'N/A'
        print(f'第{i}条数据没有找到政策文本内容')
    # # 打印提取的政策文本内容
    # print(policy_text)
    #保存到policy_data
    policy_data.append([title,theme, issuing_agency, joint_issuing_agency, implementation_date, date_formed, abolition_date,document_number_1, date_published, validity, document_number, policy_text])




-------------------------------------------第1975条数据----------------------------------------
标题：您访问的页面已撤稿或删除_首都之窗_北京市人民政府门户网站 主题分类：N/A 发文机构：N/A 联合发文机构：N/A 实施日期：N/A 形成日期：N/A 废止日期：N/A 发文字号:N/A 发布日期：N/A 有效性：N/A 文件来源：N/A 发文字号：
第1975条数据没有找到政策文本内容
-------------------------------------------第1976条数据----------------------------------------
标题：北京市绿化条例 主题分类：N/A 发文机构：N/A 联合发文机构：N/A 实施日期：N/A 形成日期：N/A 废止日期：N/A 发文字号:N/A 发布日期：N/A 有效性：N/A 文件来源：N/A 发文字号：
-------------------------------------------第1977条数据----------------------------------------
标题：北京市人民政府关于划定禁止使用高排放非道路移动机械区域的通告 主题分类：城乡建设、环境保护/环境监测、保护与治理 发文机构：北京市人民政府 联合发文机构： 实施日期：2019-08-01 形成日期：2019-07-25 废止日期：2021-12-01 发文字号:京政发〔2019〕10号 发布日期：2019-07-26 有效性：是 文件来源：政府公报 2019年 第32期(总第620期) 发文字号：京政发〔2019〕10号
-------------------------------------------第1978条数据----------------------------------------
标题：北京市农业机械化促进条例 主题分类：N/A 发文机构：N/A 联合发文机构：N/A 实施日期：N/A 形成日期：N/A 废止日期：N/A 发文字号:N/A 发布日期：N/A 有效性：N/A 文件来源：N/A 发文字号：
-------------------------------------------第19

In [42]:
# 转换成dataframe
policy_df_1 = pd.DataFrame(policy_data, columns=['标题','主题分类', '发文机构', '联合发文机构', '实施日期', '形成日期', '废止日期', '发文字号','发布日期', '有效性', '发文字号', '政策文本'])
policy_df_1

,标题,主题分类,发文机构,联合发文机构,实施日期,形成日期,废止日期,发文字号,发布日期,有效性,发文字号,政策文本
0,北京市实施《中华人民共和国动物防疫法》办法,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,,（2024年7月26日北京市第十六届人民代表大会常务委员会第十一次会议通过）\n 第一条 ...
1,北京市人力资源和社会保障局关于2024年积分落户公示及落户办理有关工作的通告,劳动、人事、监察/人事工作,北京市人力资源和社会保障局,,,2024-07-15,,京人社发〔2024〕11号,2024-07-15,是,京人社发〔2024〕11号,京人社发〔2024〕11号\n 根据《北京市积分落户管理办法》（京政办发〔2020〕9号）...
2,北京市人力资源和社会保障局等部门关于印发《北京市加快数字人才培育支撑数字经济发展实施方案（2...,劳动、人事、监察/人事工作,北京市人力资源和社会保障局,北京市人才工作局;中共北京市委网络安全和信息化委员会办公室;北京市发展和改革委员会;北京市教...,,2024-07-05,,京人社专技字〔2024〕104号,2024-07-11,是,京人社专技字〔2024〕104号,京人社专技字〔2024〕104号\n各区党委组织部、网信办，各区人力资源社会保障局、发展改革...
3,北京市民政局关于印发《北京市街道（乡镇）区域养老服务中心内设社区餐厅建设工作指引》的通知,民政、扶贫、救灾/其他,北京市民政局,,,2024-07-01,,京民养老发〔2024〕74号,2024-07-08,是,京民养老发〔2024〕74号,京民养老发〔2024〕74号\n各区民政局，北京经济技术开发区社会事业局：\n 为贯彻落实...
4,北京市司法局关于参加2023年国家统一法律职业资格考试成绩合格的2024年应届毕业生等在京申...,劳动、人事、监察/人事工作,北京市司法局,,,2024-07-05,,〔〕号,2024-07-05,是,〔〕号,根据《中华人民共和国司法部公告（第5号）》，现就参加2023年国家统一法律职业资格考试成绩合...
...,...,...,...,...,...,...,...,...,...,...,...,...
1969,北京市科学技术奖励办法,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,,第一条\n 为了奖励在本市科学技术进步活动中作出突出贡献的个人和组织，调动科学技术工作者的积...
1970,北京市住房和城乡建设委员会关于印发《中央单位在北京地区建设项目登记备案管理办法》的通知,城乡建设、环境保护/城乡建设（含住房）,北京市住房和城乡建设委员会,,2019-08-12,2019-07-29,,京建法〔2019〕15号,2019-08-06,是,京建法〔2019〕15号,京建法〔2019〕15号\n各有关单位：\n 经市政府同意，现将《中央单位在北京地区建设项...
1971,北京市公园条例,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,,(2002年10月17日北京市第十一届人民代表大会常务委员会第三十七次会议通过 根据2019...
1972,北京市园林绿化局关于印发《本市老旧小区绿化改造基本要求》的通知,城乡建设、环境保护/环境监测、保护与治理,北京市园林绿化局,,,2019-07-17,,京绿办发〔2019〕139号,2019-07-29,是,京绿办发〔2019〕139号,京绿办发〔2019〕139号\n各区园林绿化局：\n 为进一步加强本市绿化薄弱老旧小区提升...


In [43]:
# 保存policy_df_1成csv文件
policy_df_1.to_csv('/home/bld/data/data3/peng/Bridge/Beijing/北京市政策文本.csv', index=False)

In [46]:
# 转换成dataframe
policy_df_2 = pd.DataFrame(policy_data, columns=['标题','主题分类', '发文机构', '联合发文机构', '实施日期', '形成日期', '废止日期', '发文字号','发布日期', '有效性', '发文字号', '政策文本'])
policy_df_2

,标题,主题分类,发文机构,联合发文机构,实施日期,形成日期,废止日期,发文字号,发布日期,有效性,发文字号,政策文本
0,您访问的页面已撤稿或删除_首都之窗_北京市人民政府门户网站,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,,N/A
1,北京市绿化条例,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,,(2009年11月20日北京市第十三届人民代表大会常务委员会第十四次会议通过 根据2016年...
2,北京市人民政府关于划定禁止使用高排放非道路移动机械区域的通告,城乡建设、环境保护/环境监测、保护与治理,北京市人民政府,,2019-08-01,2019-07-25,2021-12-01,京政发〔2019〕10号,2019-07-26,是,京政发〔2019〕10号,京政发〔2019〕10号\n 为进一步减少非道路移动机械污染物排放，持续改善本市空气质量，...
3,北京市农业机械化促进条例,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,,(2010年12月23日北京市第十三届人民代表大会常务委员会第二十二次会议通过 根据2016...
4,北京市湿地保护条例,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,,(2012年12月27日北京市第十三届人民代表大会常务委员会第三十七次会议通过 根据2019...
...,...,...,...,...,...,...,...,...,...,...,...,...
4346,关于召开整顿社会秩序会议情况的报告,京政办发〔1980〕6号,1980-01-09,,N/A,N/A,N/A,N/A,N/A,N/A,京政办发〔1980〕6号,N/A
4347,北京市人民政府办公厅\n转发教育部等六部委《关于拟改变\n对出国留学人员、访问学者所得奖学金...,京政办发〔1980〕5号,1980-01-09,,N/A,N/A,N/A,N/A,N/A,N/A,京政办发〔1980〕5号,N/A
4348,北京市人民政府办公厅转发《财政部、外交部、国务院办公室关于修订三项外事费用开支标准的通知》,京政办发〔1980〕1号,1980-01-07,,N/A,N/A,N/A,N/A,N/A,N/A,京政办发〔1980〕1号,N/A
4349,关于将北京市公共交通局\n改为北京市公共交通总公司的通知,京政发〔1980〕38号,1980-01-01,,N/A,N/A,N/A,N/A,N/A,N/A,京政发〔1980〕38号,N/A


In [47]:
# 将policy_df_1与policy_df_2合并
policy_df = pd.concat([policy_df_1, policy_df_2], axis=0)
policy_df

,标题,主题分类,发文机构,联合发文机构,实施日期,形成日期,废止日期,发文字号,发布日期,有效性,发文字号,政策文本
0,北京市实施《中华人民共和国动物防疫法》办法,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,,（2024年7月26日北京市第十六届人民代表大会常务委员会第十一次会议通过）\n 第一条 ...
1,北京市人力资源和社会保障局关于2024年积分落户公示及落户办理有关工作的通告,劳动、人事、监察/人事工作,北京市人力资源和社会保障局,,,2024-07-15,,京人社发〔2024〕11号,2024-07-15,是,京人社发〔2024〕11号,京人社发〔2024〕11号\n 根据《北京市积分落户管理办法》（京政办发〔2020〕9号）...
2,北京市人力资源和社会保障局等部门关于印发《北京市加快数字人才培育支撑数字经济发展实施方案（2...,劳动、人事、监察/人事工作,北京市人力资源和社会保障局,北京市人才工作局;中共北京市委网络安全和信息化委员会办公室;北京市发展和改革委员会;北京市教...,,2024-07-05,,京人社专技字〔2024〕104号,2024-07-11,是,京人社专技字〔2024〕104号,京人社专技字〔2024〕104号\n各区党委组织部、网信办，各区人力资源社会保障局、发展改革...
3,北京市民政局关于印发《北京市街道（乡镇）区域养老服务中心内设社区餐厅建设工作指引》的通知,民政、扶贫、救灾/其他,北京市民政局,,,2024-07-01,,京民养老发〔2024〕74号,2024-07-08,是,京民养老发〔2024〕74号,京民养老发〔2024〕74号\n各区民政局，北京经济技术开发区社会事业局：\n 为贯彻落实...
4,北京市司法局关于参加2023年国家统一法律职业资格考试成绩合格的2024年应届毕业生等在京申...,劳动、人事、监察/人事工作,北京市司法局,,,2024-07-05,,〔〕号,2024-07-05,是,〔〕号,根据《中华人民共和国司法部公告（第5号）》，现就参加2023年国家统一法律职业资格考试成绩合...
...,...,...,...,...,...,...,...,...,...,...,...,...
4346,关于召开整顿社会秩序会议情况的报告,京政办发〔1980〕6号,1980-01-09,,N/A,N/A,N/A,N/A,N/A,N/A,京政办发〔1980〕6号,N/A
4347,北京市人民政府办公厅\n转发教育部等六部委《关于拟改变\n对出国留学人员、访问学者所得奖学金...,京政办发〔1980〕5号,1980-01-09,,N/A,N/A,N/A,N/A,N/A,N/A,京政办发〔1980〕5号,N/A
4348,北京市人民政府办公厅转发《财政部、外交部、国务院办公室关于修订三项外事费用开支标准的通知》,京政办发〔1980〕1号,1980-01-07,,N/A,N/A,N/A,N/A,N/A,N/A,京政办发〔1980〕1号,N/A
4349,关于将北京市公共交通局\n改为北京市公共交通总公司的通知,京政发〔1980〕38号,1980-01-01,,N/A,N/A,N/A,N/A,N/A,N/A,京政发〔1980〕38号,N/A


In [48]:
# 保存policy_df成csv文件
policy_df.to_csv('/home/bld/data/data3/peng/Bridge/Beijing/北京市政策文本_6326.csv', index=False)

In [36]:
data[228:229]

,标题,标题链接,标签,信息,信息1,信息2
228,北京市人民代表大会常务委员会关于加强国有资产管理情况监督的决定,https://www.beijing.gov.cn/zhengce/dfxfg/20230...,北京市,[发文字号] 北京市人民代表大会常务委员会公告〔十六届〕第7号,[发文机构] 北京市人民代表大会常务委员会,[发布日期] 2023-09-23
